In [ ]:
#!pip install mysql-connector-python

import os
import pandas as pd
import mysql.connector
from  mysql.connector import errorcode
import ast

In [ ]:
def limpieza_datos_guionistas_pelis():
    """
    Esta función transforma un csv en una lista de tuplas de guionistas por pelicula.
    
    Args:
    - No recibe parametros.

    Returns:
    - devuelve una lista de tuplas (id pelicula, nombre guionista).
    """
    
    lista_tuplas_guionistas =[]
    lista_final_guionistas = []

    for i in range(1990,2024):
    
        try:
            df_pelis = pd.read_csv(f'datos/datos_peliculas/lista_peliculas_{i}_3000.csv')
        except:
            print(f"lista_peliculas_{i}_1000.csv - no existe")            

        orden_guion = ["0","8"]

        df_guionistas_pelis = df_pelis.reindex(columns = orden_guion)

        # tupla guionistas
        for fila in df_guionistas_pelis.values:

            try:
                linea = ast.literal_eval(fila[1])
            except:
                linea = 'No data'   

            for elemento in linea:
                lista_tuplas_guionistas.append((fila[0], elemento))

    df_guionistas = pd.DataFrame(set(lista_tuplas_guionistas), columns =['id', 'nombre'])


    #quitar nan y null del dataframe
    df_final_guionistas = df_guionistas.dropna(how='any', axis=0) 

    for fila in df_final_guionistas.values:
        lista_final_guionistas.append(tuple(fila))

    return lista_final_guionistas    

In [ ]:
def insertar_guionistas_pelis(lista):
    """
    Esta función toma una lista de tuplas y la inserta en una tabla de la bbdd
    
    Args:
    - lista: listade tuplas donde cada tupla incluye los datos de un guionista: id pelicula, nombre y apellido del guionista

    Returns:
    - esta funcion devuelve el número de datos insertados

    """
    contrasena = os.environ.get('contraseña')

    #conexion     
    cnx = mysql.connector.connect(user="root",
                            password=contrasena,
                            host="localhost", database = "bho_peliculas_3")

    mycursor = cnx.cursor()

    #tabla guionistas por pelicula
    sql_guionistas_peliculas = "INSERT INTO guionistas_peliculas (id_pelicula, nombre_guionista) VALUES (%s, %s)"

    try:
        mycursor.executemany(sql_guionistas_peliculas, lista)
        cnx.commit()
        print(mycursor.rowcount, "registros insertados")

    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)
        print("peliculas - error inserción")    

    cnx.close()    

    return mycursor.rowcount


In [ ]:
#llamar a las funciones para limpiar los datos e insertarlos en la bbdd
insertar_guionistas_pelis(limpieza_datos_guionistas_pelis())